In [7]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [9]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [10]:
import os
import pandas as pd
from datetime import datetime, timezone

from configs import config
from utils.custom_classes.data_loader import CompasDataset
from utils.metrics_computation_interfaces import compute_metrics_multiple_runs

## Configs

In [11]:
DATASET_NAME = "COMPAS"

# RUNS_SEED_LST = [101, 111, 201, 221, 311, 333, 410, 555, 666, 851, 999, 1010]
RUNS_SEED_LST = [100, 200]
SEX_priv = 0
RACE_priv = 'Caucasian'
N_ESTIMATORS = 100
SENSITIVE_ATTRIBUTES = ['sex', 'race']
PRIV_VALUES = [SEX_priv, RACE_priv]
TUNED_PARAMS_FILE_PATH = os.path.join('results', 'models_tuning', f'tuning_results_{DATASET_NAME}_20230117__012024.csv')
SAVE_RESULTS_DIR_PATH = os.path.join('results', 'hypothesis_space', f'{DATASET_NAME}_Metrics_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}')

## Models tuned hyper-parameters

In [12]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,COMPAS_Without_Sensitive_Attributes,LogisticRegression,0.6785,0.6837,"{'C': 1, 'max_iter': 50, 'penalty': 'l2', 'sol..."
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,0.6952,0.6989,"{'criterion': 'gini', 'max_depth': 20, 'max_fe..."
2,COMPAS_Without_Sensitive_Attributes,RandomForestClassifier,0.6962,0.7008,"{'max_depth': 4, 'max_features': 0.6, 'min_sam..."
3,COMPAS_Without_Sensitive_Attributes,XGBClassifier,0.6971,0.7008,"{'lambda': 100, 'learning_rate': 0.1, 'max_dep..."
4,COMPAS_Without_Sensitive_Attributes,KNeighborsClassifier,0.7056,0.7112,"{'metric': 'minkowski', 'n_neighbors': 25, 'we..."
5,COMPAS_Without_Sensitive_Attributes,MLPClassifier,0.6834,0.6875,"{'activation': 'relu', 'hidden_layer_sizes': (..."


## Load dataset

In [13]:
dataset = CompasDataset(dataset_path='data/COMPAS.csv')
dataset.X_data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,race,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M,sex
0,25,0.0,-2.340451,1.0,-15.010999,African-American,1,0,0,0,1,1
1,26,0.0,0.000000,0.0,0.000000,Caucasian,1,0,0,1,0,0
2,21,0.0,0.000000,0.0,0.000000,Caucasian,0,0,1,1,0,1
3,29,0.0,0.000000,0.0,6.000000,African-American,1,0,0,0,1,1
4,40,0.0,0.000000,0.0,7.513697,Caucasian,1,0,0,1,0,1


## Run experiments

In [14]:
multiple_runs_metrics_dct = compute_metrics_multiple_runs(dataset, config.TEST_SET_FRACTION, DATASET_NAME,
                                                          config, models_tuned_params_df, N_ESTIMATORS, SENSITIVE_ATTRIBUTES, PRIV_VALUES,
                                                          RUNS_SEED_LST, SAVE_RESULTS_DIR_PATH, debug_mode=False)

  0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze DecisionTreeClassifier  ##############################
Baseline X_train shape:  (4222, 12)
Baseline X_test shape:  (1056, 12)


2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 100
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 100 was tested
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 100
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 100 was tested
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 100
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 100 was tested
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 100
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 100 was tested
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 100
2023-01-23 16:30:14 abstract_overall_variance_analyzer.py INFO    : Classifier 5 / 100 w



##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.6752
Mean: 0.5295
Std: 0.0966
IQR: 0.1302
Entropy: 0.0
Jitter: 0.1816
Per sample accuracy: 0.6593
Label stability: 0.7387




100%|██████████| 1/1 [00:12<00:00, 12.16s/it]


  0%|          | 0/1 [00:00<?, ?it/s]2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 100
2023-01-23 16:30:26 abstract_overall_variance_analy

##############################  [Model 1 / 1] Analyze DecisionTreeClassifier  ##############################
Baseline X_train shape:  (4222, 12)
Baseline X_test shape:  (1056, 12)


2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 16 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 17 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 17 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 18 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 18 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 19 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 19 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 20 / 100
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Classifier 20 / 100 was tested
2023-01-23 16:30:26 abstract_overall_variance_analyzer.py INFO    : Start testing of 



##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.6847
Mean: 0.5207
Std: 0.0927
IQR: 0.1192
Entropy: 0.0
Jitter: 0.1602
Per sample accuracy: 0.6606
Label stability: 0.7793




100%|██████████| 1/1 [00:11<00:00, 12.00s/it]